In [ ]:
import intake
import pandas as pd

df = intake.open_csv('./data/bird_migration/{species}.csv').read()

def fill_day(v):
    next_year = v.assign(day=v.day + v.day.max())
    last_year = v.assign(day=v.day - v.day.max())
    surrounding_years = pd.concat([last_year, v, next_year])
    filled = surrounding_years.assign(
        lat=surrounding_years.lat.interpolate(), 
        lon=surrounding_years.lon.interpolate())
    this_year = filled[filled.day.isin(v.day)]
    return this_year

df = pd.concat([fill_day(v) for k, v in df.groupby('species')])

colors = pd.read_csv('./assets/colormap.csv', header=None, names=['R', 'G', 'B'])
species_cmap = dict(zip(df.species.cat.categories, 
                        ['#{row.R:02x}{row.G:02x}{row.B:02x}'.format(row=row) 
                         for _, row in colors.iterrows()]))

## Geoviews

`geoviews` is the geographic equivalent of `holoviews` but with some added metadata for geographic information. 

In [ ]:
import holoviews as hv
import geoviews as gv

import geoviews.tile_sources as gts
import cartopy.crs as ccrs
hv.extension('bokeh')

One of the most important extra bits of information is the *Coordinate Reference System*. This is how we can set up a `geoviews.Dataset` of our data.

In [ ]:
bird_ds = gv.Dataset(df, kdims=['lon', 'lat'], vdims=['day', 'species'], crs=ccrs.PlateCarree())
bird_ds

### Visualizing data

In [ ]:
%%opts Points [color_index='day' height=500 width=400 show_legend=False] (size=1, cmap='colorwheel')

p = bird_ds.to(gv.Points)
p * gv.feature.coastline

In [ ]:
print(p)

### Grouping

We can group the data in the same way that we did for `holoviews`. To save time we can set `dynamic=True` to render on the fly.

In [ ]:
grouped_birds = p.groupby('day', dynamic=True)
print(grouped_birds)

In [ ]:
%%opts Points [color_index='species' height=500 width=400 show_legend=False, tools=['hover', 'tap', 'box_select']] 
%%opts Points (cmap=species_cmap size=5)

grouped_birds * gv.feature.coastline

Hmmm but that is out of bounds. We can't set boundaries on coastlines so let's put that over tiles instead.

In [ ]:
tiles = gts.EsriImagery()
tiles.extents = df.lon.min(), df.lat.min(), df.lon.max(), df.lat.max()

In [ ]:
styled_birds = grouped_birds.options(color_index='species', height=500, width=400, 
                                     show_legend=False, tools=['hover', 'tap', 'box_select'], 
                                     cmap=species_cmap, size=5)

In [ ]:
tiles * styled_birds

### Equivalent in hvplot

It is straightforward to use geoviews with `hvplot` as well. All we need is to set the `geo` kwarg to `True`.

In [ ]:
import hvplot.pandas

In [ ]:
df.hvplot.points(x='lon', y='lat', groupby='day', color='species', 
                 cmap=species_cmap, legend=False, geo=True,
                 height=700, width=600).options(tools=['hover', 'tap', 'box_select']) * tiles

## Adding another layer

Now let's put it over our air temperature data. We'll set it up the same way as before, but we'll include `geo=True` in the arguments.

In [ ]:
import xarray as xr
import hvplot.xarray

data_url = 'http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/ncep/air.day.ltm.nc'

# I downloaded the file locally because I was hitting rate limits. Just comment out this line
data_url = './data/air.day.ltm.nc'

ds = xr.open_dataset(data_url)
ds = ds.rename(time='day').sel(level=1000)
ds['day'] = list(range(1,366))

In [ ]:
grouped_air = ds.hvplot('lon', 'lat', groupby='day', geo=True, height=600)
grouped_air * styled_birds * gv.feature.coastline

[Next Section](./04_panel.ipynb#panel)